In [68]:
import sqlite3 as dbms

conn = dbms.connect("students.sqlite3")  # Как оригинально!
cursor = conn.cursor()

In [69]:
with open("11.drop_ddl.sql", 'r', encoding='utf-8') as f:
    drop_ddl = f.read()
with open("11.create_ddl.sql", 'r', encoding='utf-8') as f:
    create_ddl = f.read()

In [70]:
if True:
    cursor.executescript(drop_ddl)
    conn.commit()
if True:
    cursor.executescript(create_ddl)
    conn.commit()

In [71]:
import sqlalchemy
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

DeclBase = declarative_base()

engine = sqlalchemy.create_engine('sqlite:///students.sqlite3', echo=False)  # echo=True для логгинга
Session = sessionmaker(bind=engine)
session = Session()

In [72]:
null=True
class Program(DeclBase):
    __tablename__ = 'programs'
    id = Column(Integer, primary_key=True)
    name = Column(String)

    students = relationship("Student", backref="program")
    programs_courses = relationship("Program_course", backref="program")

    def __init__(self, name):
        self.name = name

class Student(DeclBase):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    card = Column(String)
    surname = Column(String)
    name = Column(String)
    patronymic = Column(String)
    
    program_id = Column(Integer, ForeignKey('programs.id'))
    marks = relationship("Mark", backref="student")
    
    
    def __init__(self, card, surname, name, patronymic, program):
        self.card = card
        self.surname = surname
        self.name = name
        self.patronymic = patronymic
        self.program = program
        
class Course(DeclBase):
    __tablename__ = 'courses'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    programs_courses = relationship("Program_course", backref="course")
    marks = relationship("Mark", backref="course")
    
    def __init__(self, name):
        self.name = name
        
class Program_course(DeclBase):
    __tablename__ = 'programs_courses'
    semester_number = Column(Integer, primary_key=True)
    
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    program_id = Column(Integer, ForeignKey('programs.id'), primary_key=True)
    
    def __init__(self, semester_number, course, program):
        self.semester_number = semester_number
        self.course = course
        self.program = program
        
class Mark(DeclBase):
    __tablename__ = 'marks'
    mark = Column(Integer)
    
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    student_id = Column(Integer, ForeignKey('students.id'), primary_key=True)
    
    def __init__(self, mark, student, course):
        self.mark = mark
        self.student = student
        self.course = course

In [73]:
se_1 = Program("Физическая физика")
se_2 = Program("Программерская прога")

st1 = Student("002004", "Алексеев", "Алексей", "Алексеевич", se_1)
st2 = Student("002003", "Гурулева", "Наталья", "Олеговна", se_1)
st3 = Student("022222", "Масютин", "Алеша", "Лексеевич", se_2)
st4 = Student("002005", "Алексеев", "Алексей", "Алексеевич", se_2)
c1 = Course("БЖД")
c2 = Course("Философия")
pc1 = Program_course("2", c1, se_1)
m1_1 = Mark("5", st1, c1)
m1_2 = Mark("7", st1, c2)
m2_1 = Mark("8", st2, c1)
m2_2 = Mark("10", st2, c2)
m3_1 = Mark("10", st3, c1)
m3_2 = Mark("5", st3, c2)
m4_1 = Mark("6", st4, c1)
m4_2 = Mark("6", st4, c2)





session.add_all([st1, st2, st3, c1, c2, pc1, m1_1, m1_2, m2_1, m2_2, m3_1, m3_2])
session.add_all([st4, m4_1, m4_2]) 
session.commit()

In [74]:
import sys

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ",s.surname, s.name, s.patronymic)
    for s in p.programs_courses:
        print("Номер семестра:", s.semester_number)
for i in session.query(Program):
    for t in i.programs_courses:
        for p in session.query(Student):
            print("Студент: ", p.surname)
            for s in p.marks:
                print("- ", s.mark, " ")

Программы и студенты
Программа:  Физическая физика
-  Алексеев Алексей Алексеевич
-  Гурулева Наталья Олеговна
Номер семестра: 2
Программа:  Программерская прога
-  Масютин Алеша Лексеевич
-  Алексеев Алексей Алексеевич
Студент:  Алексеев
-  5  
-  7  
Студент:  Гурулева
-  8  
-  10  
Студент:  Масютин
-  5  
-  10  
Студент:  Алексеев
-  6  
-  6  
